In [73]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import torch.nn as nn
from transformers import BertModel
import math
# importing required libraries
import torch.nn.functional as F
import math,copy,re
import warnings
import pandas as pd
# import seaborn as sns
import torchtext
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import random
warnings.simplefilter("ignore")
print(torch.__version__)

2.1.1+cu121


In [81]:
class BERTSentenceEncoder:
    def __init__(self, model_name='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)

    def encode_sentences(self, input_sentences):
        tokenized_input = self.tokenizer(input_sentences, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**tokenized_input)
        encoded_sentences = outputs.last_hidden_state
        return encoded_sentences

In [82]:
input_sentences = ["Your first sentence", "Your second sentence at the test"]
tmp = BERTSentenceEncoder()
encoded_sentences = tmp.encode_sentences(input_sentences)

In [83]:
encoded_sentences.shape

torch.Size([2, 8, 768])

In [97]:
out.shape

torch.Size([200, 8, 768])

In [98]:
class CustomTransformerDecoder(nn.Module):
    def __init__(self, input_size, output_size, num_layers, num_heads, hidden_size, max_length=200):
        super(CustomTransformerDecoder, self).__init__()

        self.embedding = nn.Linear(input_size, hidden_size)
        self.positional_encoding = PositionalEncoding(hidden_size, max_length)
        decoder_layers = nn.TransformerDecoderLayer(d_model=hidden_size, nhead=num_heads)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layers, num_layers=num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        x = self.positional_encoding(x)
        x = self.transformer_decoder(x)
        x = self.fc(x)
        return x

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=200):
        super(PositionalEncoding, self).__init__()

        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pos_enc = torch.zeros((1, max_len, d_model))

        pos_enc[0, :, 0::2] = torch.sin(position * div_term)
        pos_enc[0, :, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pos_enc', pos_enc)

    def forward(self, x):
        return x + self.pos_enc[:, :x.size(1), :]

In [99]:
# Example usage
input_size = 768
output_size = 109
num_layers = 6
num_heads = 8
hidden_size = 256  # Adjust this based on your specific needs
max_length = 200

# Instantiate the custom transformer decoder
custom_decoder = CustomTransformerDecoder(input_size, output_size, num_layers, num_heads, hidden_size, max_length)

# Example input tensor with shape [2, 8, 768]
input_tensor = encoded_sentences

# Forward pass
output_tensor = custom_decoder(input_tensor)

print("Input shape:", input_tensor.shape)
print("Output shape:", output_tensor.shape)

TypeError: TransformerDecoder.forward() missing 1 required positional argument: 'memory'